<a href="https://colab.research.google.com/github/Shahid-TheMatrixMaker/100-Data-Science-Project/blob/main/House%2BPrices%2BPrediction%2BUsing%2BAdvanced%2BRegression%2BTechniques.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Importing the needed libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import string
import nltk
from nltk.corpus import stopwords #for cleaning
from nltk.stem import LancasterStemmer ##for cleaning
from scipy.sparse import csr_matrix
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import r2_score
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings("ignore")



In [ ]:
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.info()

In [ ]:
test_data.info()

In [ ]:
sns.displot(train_data['SalePrice'])

In [ ]:
corr = train_data.corr()['SalePrice']
print(corr.sort_values())

In [ ]:
missing = train_data.isnull().sum()
print(missing)

In [ ]:
# Calculate the first quartile of the 'SalePrice' column
Q1 = train_data['SalePrice'].quantile(0.25)
# Calculate the third quartile of the 'SalePrice' column
Q3 = train_data['SalePrice'].quantile(0.75)
# Calculate the interquartile range of the 'SalePrice' column
IQR = Q3 - Q1
# Identify the outliers in the 'SalePrice' column
outliers = train_data[(train_data['SalePrice'] < (Q1 - 1.5 * IQR)) | (train_data['SalePrice'] > (Q3 + 1.5 * IQR))]

In [ ]:
# Drop the outliers from the 'train_data' DataFrame
train_data = train_data.drop(outliers.index)

In [ ]:
numeric_columns_with_null = train_data.select_dtypes(include=['number']).columns[
    train_data.select_dtypes(include=['number']).isnull().any()]

object_columns_with_null = train_data.select_dtypes(exclude=['number']).columns[
    train_data.select_dtypes(exclude=['number']).isnull().any()]

In [ ]:
print(numeric_columns_with_null)

In [ ]:
print(object_columns_with_null)

In [ ]:
train_data["Alley"]= train_data["Alley"].fillna("No alley")
train_data["MasVnrType"]= train_data["MasVnrType"].fillna("unknown")
train_data["BsmtQual"]= train_data["BsmtQual"].fillna("No Basement")
train_data["BsmtCond"]= train_data["BsmtCond"].fillna("No Basement")
train_data["BsmtExposure"]= train_data["BsmtExposure"].fillna("No Basement")
train_data["BsmtFinType1"]= train_data["BsmtFinType1"].fillna("No Basement")
train_data["BsmtFinType2"]= train_data["BsmtFinType2"].fillna("No Basement")
train_data["Electrical"]= train_data["Electrical"].fillna("unknown")
train_data["FireplaceQu"]= train_data["FireplaceQu"].fillna("No Fireplace'")
train_data["GarageType"]= train_data["GarageType"].fillna("No Garage")
train_data["GarageFinish"]= train_data["GarageFinish"].fillna("No Garage")
train_data["GarageQual"]= train_data["GarageQual"].fillna("No Garage")
train_data["GarageCond"]= train_data["GarageCond"].fillna("No Garage")
train_data["PoolQC"]= train_data["PoolQC"].fillna("No Pool")
train_data["Fence"]= train_data["Fence"].fillna("No Fence")
train_data["MiscFeature"]= train_data["MiscFeature"].fillna("unknown")

In [ ]:
test_data["Alley"]= test_data["Alley"].fillna("No alley")
test_data["MasVnrType"]= test_data["MasVnrType"].fillna("unknown")
test_data["BsmtQual"]= test_data["BsmtQual"].fillna("No Basement")
test_data["BsmtCond"]= test_data["BsmtCond"].fillna("No Basement")
test_data["BsmtExposure"]= test_data["BsmtExposure"].fillna("No Basement")
test_data["BsmtFinType1"]= test_data["BsmtFinType1"].fillna("No Basement")
test_data["BsmtFinType2"]= test_data["BsmtFinType2"].fillna("No Basement")
test_data["Electrical"]= test_data["Electrical"].fillna("unknown")
test_data["FireplaceQu"]= test_data["FireplaceQu"].fillna("No Fireplace'")
test_data["GarageType"]= test_data["GarageType"].fillna("No Garage")
test_data["GarageFinish"]= test_data["GarageFinish"].fillna("No Garage")
test_data["GarageQual"]= test_data["GarageQual"].fillna("No Garage")
test_data["GarageCond"]= test_data["GarageCond"].fillna("No Garage")
test_data["PoolQC"]= test_data["PoolQC"].fillna("No Pool")
test_data["Fence"]= test_data["Fence"].fillna("No Fence")
test_data["MiscFeature"]= test_data["MiscFeature"].fillna("unknown")

In [ ]:
test_data.head()

In [ ]:
# Fill the numeric Nan values
numeric_columns = ["LotFrontage", "MasVnrArea", "GarageYrBlt"]
numeric_imputer = SimpleImputer(strategy='mean')

train_data[numeric_columns] = numeric_imputer.fit_transform(train_data[numeric_columns])
test_data[numeric_columns] = numeric_imputer.transform(test_data[numeric_columns])

In [ ]:
Object1_columns = train_data.select_dtypes(include=['object']).columns
Object2_columns = test_data.select_dtypes(include=['object']).columns
label_encoder = LabelEncoder()

for column in Object1_columns:
    print(column)
    train_data[column] = label_encoder.fit_transform(train_data[column])

for column in Object2_columns:
    print(column)
    test_data[column] = label_encoder.fit_transform(test_data[column])

In [ ]:
# Feature Engineering and Multicollinearity Check
# Feature Engineering:Creating a new feature by combining highly correlated features
train_data["TotalSF"] = train_data["1stFlrSF"] + train_data["2ndFlrSF"] + train_data["TotalBsmtSF"]
test_data["TotalSF"] = test_data["1stFlrSF"] + test_data["2ndFlrSF"] + test_data["TotalBsmtSF"]

# Example: Creating a new feature by calculating the age of the house
train_data["HouseAge"] = train_data["YrSold"] - train_data["YearBuilt"]
test_data["HouseAge"] = test_data["YrSold"] - test_data["YearBuilt"]

# Multicollinearity Check - Variance Inflation Factor (VIF):

from statsmodels.stats.outliers_influence import variance_inflation_factor

# Select the features you want to check for multicollinearity
selected_features = ["TotalSF", "HouseAge", "OverallQual", "OverallCond"]

# Calculate VIF scores for the selected features
vif = pd.DataFrame()
vif["Feature"] = selected_features
vif["VIF"] = [variance_inflation_factor(train_data[selected_features].values, i) for i in range(len(selected_features))]

# Display the VIF scores
print(vif)

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x="MSZoning", data=train_data, palette="pastel")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.countplot(x="MSSubClass", data=train_data, palette="YlGnBu")
plt.show()

In [ ]:
plt.figure(figsize=(10, 8))
sns.countplot(x="SaleCondition", data=train_data, palette="pastel")
plt.yticks(np.arange(0, 2500, 100))
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
kind_counts = train_data["LotShape"].value_counts()
plt.pie(kind_counts, labels=kind_counts.index, autopct="%0.0f%%")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
kind_counts = train_data["LandContour"].value_counts()
plt.pie(kind_counts, labels=kind_counts.index, autopct="%0.0f%%")
plt.show()

In [ ]:
plt.figure(figsize=(8, 6))
sns.histplot(train_data['SalePrice'], bins=30, kde=True, color='#76698a')
plt.show()

In [ ]:
X_train = train_data.drop(columns=["SalePrice", "Id"])
y_train  = train_data["SalePrice"]

In [ ]:
model = LinearRegression()
model.fit(X_train ,y_train)

In [ ]:
model.score(X_train ,y_train)

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(model, X_train, y_train, cv=5)
print(scores)

In [ ]:
from sklearn.linear_model import Ridge
from sklearn.model_selection import GridSearchCV

# Define the model and parameters
model = Ridge()
parameters = {'alpha': [0.1, 1, 10]}

# Create GridSearchCV
grid_search = GridSearchCV(model, parameters, cv=5)

# Fit GridSearchCV
grid_search.fit(X_train, y_train)

# Get the best model
best_model = grid_search.best_estimator_

print(best_model)
